In [ ]:
!df -h

In [ ]:
import os
os.environ['HF_HOME'] = '/run/cache/'

In [ ]:
import pandas as pd

from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl", device_map="auto")

In [ ]:
input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

In [ ]:
test_df = pd.read_csv('../data/test.csv')

In [ ]:
test_df

In [ ]:
def llm(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(
        input_ids,
        max_length=100
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

In [ ]:
import re

def get_answer(question):
    prompt = f"""Please solve the following high school math problem step by step. Explain your reasoning clearly and provide the final answer.

{question}

Step-by-step solution and final answer:"""

    return llm(prompt)

def parse_answer(question, message):
    prompt = f""" Please analyze the solution for a given problem, fix it if needed, and then provide the final answer.         
Your response should end in the format: 'Hence, the final answer is [numeric string].
    
Q: {question}

Solution: {message}

Analysis: 
    
    """
    
    return llm(prompt)


def extract_numerical_answer(text):
    # Look for patterns like "Final answer: X" or "The answer is X" at the end of the text
    match = re.search(r'(?:final answer|the answer is)[:\s]*([+-]?\d*\.?\d+)', text, re.IGNORECASE)
    if match:
        return float(match.group(1))
    else:
        # If no clear final answer, look for the last number in the text
        numbers = re.findall(r'[+-]?\d*\.?\d+', text)
        return float(numbers[-1]) if numbers else 1.0

In [ ]:
question = test_df.iloc[0]['problem_text']
response_1 = get_answer(question)
response_1 = response_1.strip()
response_1

In [ ]:
response_2 = parse_answer(question, response_1)
response_2 = response_2.strip()
response_2

In [ ]:
numerical_answer = extract_numerical_answer(parse_answer(question, response_1))
numerical_answer

In [ ]:
from tqdm import tqdm

GLOBAL_SOLUTIONS_FOR_SUBMISSION = []

for idx, row in tqdm(test_df.iterrows(), total=len(test_df)):

    question = row['problem_text']
    response_1 = get_answer(question)
    response_1 = response_1.strip()
    
    response_2 = parse_answer(question, response_1)
    response_2 = response_2.strip()
    
    numerical_answer = extract_numerical_answer(response_2)
    
    GLOBAL_SOLUTIONS_FOR_SUBMISSION.append({
        'problem_id': row['problem_id'],
        'llm_out_1': response_1,
        'llm_out_2': response_2,
        'answer': numerical_answer
    })
    
pd.DataFrame(GLOBAL_SOLUTIONS_FOR_SUBMISSION).to_csv('baseline_answers_with_double_check.csv', index=False)

In [ ]:
pd.DataFrame(GLOBAL_SOLUTIONS_FOR_SUBMISSION)